# Data Cleaning and Writing to DB
This file is completely for preliminary data cleaning and writing to the database.

In [1]:
# import pacakges
import numpy as np
import pandas as pd

import datetime

import pymysql

In [2]:
# column names for tweets and users
tweets_cols = ["tweet_id", "tweet_time", "handle", "location", "latitude", "longitude", "tweet_text", "media",
              "favs", "retweets"]

users_cols = ["name", "handle", "language", "verified", "followers", "following", "statuses", "favourites"]

In [3]:
# reading in data
# tweets
tweets = pd.read_csv("../../Data/tweet.csv", names =  tweets_cols)

# users
users = pd.read_csv("../../Data/users.csv", names = users_cols)

In [ ]:
# tweets information
tweets.info()

Since the latitude and longitude contain very little information, it's better to drop 'em.

In [4]:
tweets.drop(["latitude", "longitude"], axis = 1, inplace = True)

tweet_id contains just the id of the tweet. tweet time is a combination of the time at and date on which the tweet was posted. Separating the two would facilitate in visualizing the tweets by time period.

In [5]:
# extracting time and date from tweets
created_at = [i[1] + "-" + i[2] + "-" + i[5] + " " + i[3] for i in tweets["tweet_time"].str.split()]

tweets["created_at"] = pd.to_datetime(pd.Series(created_at))

Now that the time and date are available, it's possible to focus on the other stuff. 

The location variable is one thing that needs to be cleaned a lot. A look at that will tell what needs to be done.

A lot of the places are redundant. "CA" stands for california. But, it's been encoded as a separate entity from USA. While this is better when it comes to visualizing tweets over location, it's better to have a single location to point to than to have multiple things for the same location.

What needs to be done is this: Get a separate country variable, and based on the name of the location, populate the country variable. For this, the geopy library will come in handy.

In [ ]:
# # the following chunk was run only once to get the coordinates.

# from geopy import geocoders
# locator = geocoders.Nominatim()

# coordinates = {"location": [], "latitude": [], "longitude": []}

# for user_location  in tweets["location"]:
#     try:
#         location = locator.geocode(user_location)
#         if location:
#             coordinates["location"].append(user_location)
#             coordinates["latitude"].append(location.latitude)
#             coordinates["longitude"].append(location.longitude)
#     except Exception as err:
#         pass

# coordinates_df = pd.DataFrame(coordinates)
# coordinates_df.to_csv("../../Data/coordinates.csv")

These coordinates will be pretty helpful in visualizing the tweet counts over locations.

The next step is to write this data to a database so that it'll be safe and accessible.

In [ ]:
# writing the cleaned files to the disk
tweets.to_csv("../../Data/tweetsCleaned.csv")

In [15]:
# converting users.verified to int
users.loc[users.verified, "verified"] = 1
users.loc[users.verified == "False", "verified"] = 0

# Writing to a database

I've created a database titled tweetsdb. The next step is to connect to it, create the tables and write the data to it. I'll use an sql script to create a database on my machine. The script can be found in src/data/.

In [6]:
pymysql.converters.encoders[np.int64] = pymysql.converters.escape_int
pymysql.converters.conversions = pymysql.converters.encoders.copy()
pymysql.converters.conversions.update(pymysql.converters.decoders)

pymysql.converters.encoders[np.datetime64] = pymysql.converters.escape_datetime
pymysql.converters.conversions = pymysql.converters.encoders.copy()
pymysql.converters.conversions.update(pymysql.converters.decoders)

In [ ]:
# creating a connection object
connection = pymysql.connect(host = "localhost", user = "pytester", password = "monty", db = "twitterdb",
                            cursorclass = pymysql.cursors.DictCursor)

In [ ]:
tweets.to_sql(name = "tweets", con = connection)

There are two tables in tweetsdb: tweets and user_info. I'll be writing the tweets to tweets and users to user_info. I'll use two separate sql statements to do this.

In [8]:
# The sql statement to get the data into the db
sql_tweets = "insert into tweets (tweet_id, tweet_time, handle, location, tweet_text, media, favs, retweets)\
values (%s, %s, %s, %s, %s, %s, %s, %s)"

sql_users = "insert into user_info (name, handle, lang, verified_account, followers_count,\
friends_count, statuses_count, favourites_count) values (%s, %s, %s, %s, %s, %s, %s, %s)"

In [7]:
# list of columns from which the info has to be fed to the table
tweet_col_nums = [0, 8, 2, 3, 4, 5, 6, 7]

To avoid errors, it's better to convert created_at to an object and then write it to the db. This way, the data can be broken down and analyzed as required using SQL.

In [9]:
tweets["created_at"] = pd.to_datetime(tweets["created_at"]).dt.strftime("%Y-%m-%d %H:%M:%S")

In [10]:
tweets.loc[tweets.location.isnull(), "location"] = None

In [11]:
# writing to the tweets table
for i in range(tweets.shape[0]):
    try:
        connection = pymysql.connect(host = "localhost", user = "pytester", password = "monty", db = "tweetsdb",
                            cursorclass = pymysql.cursors.DictCursor)
        with connection.cursor() as cursor:
            cursor.execute(sql_tweets, tweets.iloc[i, tweet_col_nums].tolist())
        connection.commit()
    except Exception as err:
        print(err)
        connection.close()

In [16]:
# writing to the user_info table:
for i in range(users.shape[0]):
    try:
        connection = pymysql.connect(host = "localhost", user = "pytester", password = "monty", db = "tweetsdb",
                            cursorclass = pymysql.cursors.DictCursor)
        with connection.cursor() as cursor:
            cursor.execute(sql_users, users.iloc[i, :].tolist())
        connection.commit()
    except Exception as err:
        print(err)
        connection.close()
    finally:
        connection.close()

With this the data have been written to the database. All required measurements are in ~src/scripts/measurements.sql file. 

For the visualizations go to src/scripts/DataAnalysis.ipynb